In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prettytable import PrettyTable
from datetime import datetime, timedelta
from tqdm import tqdm
import statsmodels.api as sm
import yfinance as yf
import numpy as np

In [19]:
IPCA=4
SELIC=12.75
ANO_DIAS_UTEIS=252

In [20]:
acoes=["KEPL3.SA","GGBR4.SA","CMIG4.SA","MGLU3.SA","ITSA4.SA","CXSE3.SA","CAML3.SA","BBDC4.SA","TAEE11.SA"]

mercado=["^BVSP"]

tickers= acoes + mercado
data = yf.download(tickers, start="2018-1-1")["Adj Close"]
data_indice=data[tickers[-1]]
data=data[tickers[:-1]]
data

[*********************100%%**********************]  10 of 10 completed


,KEPL3.SA,GGBR4.SA,CMIG4.SA,MGLU3.SA,ITSA4.SA,CXSE3.SA,CAML3.SA,BBDC4.SA,TAEE11.SA
Date,,,,,,,,,
2018-01-02,2.162874,8.705925,2.812696,2.390916,6.157748,NaN,6.456695,15.015118,12.173514
2018-01-03,2.207822,8.692501,2.788448,2.360766,6.230323,NaN,6.448437,15.084140,12.190535
2018-01-04,2.235647,9.162366,2.756118,2.417448,6.347562,NaN,6.605315,15.331638,11.997661
2018-01-05,2.252770,9.048256,2.760160,2.369811,6.358727,NaN,6.753934,15.418473,12.054389
2018-01-08,2.247420,9.363736,2.768242,2.370111,6.353144,NaN,6.745678,15.414135,11.997661
...,...,...,...,...,...,...,...,...,...
2023-10-03,11.050000,23.290001,12.090000,1.840000,8.810000,11.24,7.770000,13.910000,33.830002
2023-10-04,10.840000,23.000000,12.210000,1.890000,8.890000,11.16,7.940000,14.380000,34.250000
2023-10-05,10.910000,22.670000,12.200000,1.810000,8.920000,11.24,7.660000,14.440000,34.240002


In [21]:
retorno_diario = data.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

In [22]:
retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []

In [23]:
# vamos usar uma simulação aleatória
numero_acoes = len(acoes)
numero_carteiras = 100000



In [24]:
np.random.seed(101)
peso = np.random.random(numero_acoes)
peso /= np.sum(peso)
retorno_anual

KEPL3.SA     0.353687
GGBR4.SA     0.260133
CMIG4.SA     0.332151
MGLU3.SA     0.151352
ITSA4.SA     0.110025
CXSE3.SA     0.162009
CAML3.SA     0.098933
BBDC4.SA     0.060078
TAEE11.SA    0.202131
dtype: float64

In [25]:
# vamos fazer um for loop para preencher as lista que criamos anteriormente
for cada_carteira in range(numero_carteiras):
    # vamos dar um peso aleatório para cada ação dentro de cada carteira
    peso = np.random.random(numero_acoes)
    peso /= np.sum(peso)
    # vamos calcular o retorno das carteiras
    retorno = np.dot(peso, retorno_anual)
    # vamos calcular a volatilidade das carteiras
    volatilidade = np.sqrt(np.dot(peso.T, np.dot(cov_anual, peso)))
    # vamos calcular o índice de Sharpe de cada carteira
    sharpe = retorno / volatilidade
    # aqui nós usamos o método apend para incluir cada carteira nas listas criadas anteriormente
    sharpe_ratio.append(sharpe)
    retorno_carteira.append(retorno)
    volatilidade_carteira.append(volatilidade)
    peso_acoes.append(peso)

In [26]:
carteira = {'Retorno': retorno_carteira,
             'Volatilidade': volatilidade_carteira,
             'Sharpe Ratio': sharpe_ratio}

for contar,acao in enumerate(acoes):
    carteira[acao+' Peso'] = [Peso[contar] for Peso in peso_acoes]

# vamos transformar nosso dicionário em um dataframe
df = pd.DataFrame(carteira)

# vamos nomear as colunas do novo dataframe
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio'] + [acao+' Peso' for acao in acoes]
df = df[colunas]

In [27]:
# vamos identificar as variáveis de interesse
menor_volatilidade = df['Volatilidade'].min()
maior_sharpe = df['Sharpe Ratio'].max()

# vamos identificar os dois principais portfolios
carteira_sharpe = df.loc[df['Sharpe Ratio'] == maior_sharpe]
carteira_min_variancia = df.loc[df['Volatilidade'] == menor_volatilidade]

In [28]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T*100)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T*100)

Essa é a carteira de Mínima Variância: 
                      45289
Retorno          21.047848
Volatilidade     18.938581
Sharpe Ratio    111.137410
KEPL3.SA Peso    10.878346
GGBR4.SA Peso     6.659424
CMIG4.SA Peso     9.088063
MGLU3.SA Peso     0.019963
ITSA4.SA Peso     1.041096
CXSE3.SA Peso    24.053177
CAML3.SA Peso    11.842866
BBDC4.SA Peso     0.687671
TAEE11.SA Peso   35.729393

Essa é a carteira com maior Sharpe Ratio: 
                      12550
Retorno          26.815706
Volatilidade     22.584133
Sharpe Ratio    118.736927
KEPL3.SA Peso    24.330661
GGBR4.SA Peso    11.844253
CMIG4.SA Peso    25.262216
MGLU3.SA Peso     0.579836
ITSA4.SA Peso     1.441638
CXSE3.SA Peso     1.710526
CAML3.SA Peso     2.367875
BBDC4.SA Peso     4.091435
TAEE11.SA Peso   28.371560


In [29]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T*100)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T*100)

Essa é a carteira de Mínima Variância: 
                      45289
Retorno          21.047848
Volatilidade     18.938581
Sharpe Ratio    111.137410
KEPL3.SA Peso    10.878346
GGBR4.SA Peso     6.659424
CMIG4.SA Peso     9.088063
MGLU3.SA Peso     0.019963
ITSA4.SA Peso     1.041096
CXSE3.SA Peso    24.053177
CAML3.SA Peso    11.842866
BBDC4.SA Peso     0.687671
TAEE11.SA Peso   35.729393

Essa é a carteira com maior Sharpe Ratio: 
                      12550
Retorno          26.815706
Volatilidade     22.584133
Sharpe Ratio    118.736927
KEPL3.SA Peso    24.330661
GGBR4.SA Peso    11.844253
CMIG4.SA Peso    25.262216
MGLU3.SA Peso     0.579836
ITSA4.SA Peso     1.441638
CXSE3.SA Peso     1.710526
CAML3.SA Peso     2.367875
BBDC4.SA Peso     4.091435
TAEE11.SA Peso   28.371560
